imports

In [ ]:
import tensorflow as tf

from tensorflow.python.framework import constant_op

from tensorflow import keras

single group risk function modified losses

loss = alpha * risk + (1 - alpha) * group_risk

alpha - value between 0 and 1


In [ ]:
@keras.saving.register_keras_serializable(package='modified_loss_functions')
class MaxGroupRiskCCEv1(tf.keras.losses.Loss):

    def __init__(self,  name='MGR_Equity_CCE', alpha=1.0, number_of_classes=None):

        super().__init__()

        self.alpha = alpha

        self.number_of_classes = number_of_classes

        if self.alpha < 0 or self.alpha > 1:

          self.alpha = 1

        print('{} alpha:\t{}'.format(name,self.alpha))

    def call(self, y_true, y_pred):

        yT = y_true[:,:self.number_of_classes] # y values

        cat = y_true[:,self.number_of_classes:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = y_pred / tf.reduce_sum(y_pred, axis=-1, keepdims=True)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)


        # group risk calculation

        cal = -tf.reduce_sum(yT * tf.math.log(y_pred), axis=-1)

        cal = tf.keras.backend.reshape(cal,(-1,1))

        cat_cal = tf.multiply(cat,cal)

        cat_cal = tf.reduce_sum(cat_cal, axis=0)

        cat_cal = tf.divide(cat_cal, n_cat + epsilon_)


        max_cal = tf.reduce_max(cat_cal)

        # risk calculation

        cce = -tf.divide(tf.reduce_sum(yT * tf.math.log(y_pred)), tf.reduce_sum(yT))

        return (self.alpha * cce) + ((1-self.alpha) * max_cal)

    def get_config(self):

        return {'name':'MGR_Equity_CCE','alpha': self.alpha, 'number_of_classes':self.number_of_classes}


@keras.saving.register_keras_serializable(package='modified_loss_functions')
class AbSumDiffCCEv1(tf.keras.losses.Loss):

    def __init__(self,  name='ASD_Equity_CCE', alpha=1.0, number_of_classes=None):

        super().__init__()

        self.alpha = alpha

        self.number_of_classes = number_of_classes

        if self.alpha < 0 or self.alpha > 1:

          self.alpha = 1

        print('{} alpha:\t{}'.format(name,self.alpha))

    def call(self, y_true, y_pred):

        yT = y_true[:,:self.number_of_classes] # y values

        cat = y_true[:,self.number_of_classes:]  # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = y_pred / tf.reduce_sum(y_pred, axis=-1, keepdims=True)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)


        # group risk calculation

        cal = -tf.reduce_sum(yT * tf.math.log(y_pred), axis=-1)

        cal = tf.keras.backend.reshape(cal,(-1,1))

        cat_cal = tf.multiply(cat,cal)

        cat_cal = tf.reduce_sum(cat_cal, axis=0)

        cat_cal = tf.divide(cat_cal, n_cat + epsilon_)


        diff_cal = tf.divide(tf.reduce_sum(tf.abs(tf.subtract(tf.expand_dims(cat_cal, axis=-1), tf.expand_dims(cat_cal, axis=0)))), 2)

        # risk calculation

        cce = -tf.divide(tf.reduce_sum(yT * tf.math.log(y_pred)), tf.reduce_sum(yT))

        return (self.alpha * cce) + ((1-self.alpha) * diff_cal)

    def get_config(self):

        return {'name':'ASD_Equity_CCE','alpha': self.alpha, 'number_of_classes':self.number_of_classes}


combo group risk function modified losses

loss = alpha * risk + (1 - alpha) * (beta * group_risk_1 + (1 - beta) * group_risk_2)

alpha - value between 0 and 1

beta - value between 0 and 1


In [ ]:
@keras.saving.register_keras_serializable(package='modified_loss_functions')
class MaxAbSumDiffCCEv1(tf.keras.losses.Loss):

    def __init__(self,  name='MGR_ASD_Equity_CCE', alpha=1.0, beta=1.0, number_of_classes=None):

        super().__init__()

        self.alpha = alpha

        self.beta = beta

        self.number_of_classes = number_of_classes

        if self.alpha < 0 or self.alpha > 1:

          self.alpha = 1

        if self.beta < 0 or self.beta > 1:

          self.beta = 1

        print('{} alpha:\t{}\tbeta:\t{}'.format(name,self.alpha,self.beta))

    def call(self, y_true, y_pred):

        yT = y_true[:,:self.number_of_classes] # y values

        cat = y_true[:,self.number_of_classes:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = y_pred / tf.reduce_sum(y_pred, axis=-1, keepdims=True)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)


        # 1st group risk calculation

        cal_1 = -tf.reduce_sum(yT * tf.math.log(y_pred), axis=-1)

        cal_1 = tf.keras.backend.reshape(cal_1,(-1,1))

        cat_cal_1 = tf.multiply(cat,cal_1)

        cat_cal_1 = tf.reduce_sum(cat_cal_1, axis=0)

        cat_cal_1 = tf.divide(cat_cal_1, n_cat + epsilon_)


        max_cal = tf.reduce_max(cat_cal_1)

        # 2nd group risk calculation

        cal_2 = -tf.reduce_sum(yT * tf.math.log(y_pred), axis=-1)

        cal_2 = tf.keras.backend.reshape(cal_2,(-1,1))

        cat_cal_2 = tf.multiply(cat,cal_2)

        cat_cal_2 = tf.reduce_sum(cat_cal_2, axis=0)

        cat_cal_2 = tf.divide(cat_cal_2, n_cat + epsilon_)


        diff_cal = tf.divide(tf.reduce_sum(tf.abs(tf.subtract(tf.expand_dims(cat_cal_2, axis=-1), tf.expand_dims(cat_cal_2, axis=0)))), 2)

        # combine group risk calculation
        grp_cal = (self.beta * max_cal) + ((1-self.beta) * diff_cal)

        # risk calculation

        cce = -tf.divide(tf.reduce_sum(yT * tf.math.log(y_pred)), tf.reduce_sum(yT))


        return (self.alpha * cce) + ((1-self.alpha) * grp_cal)

    def get_config(self):

        return {'name': 'MGR_ASD_Equity_CCE', 'alpha': self.alpha, 'beta': self.beta, 'number_of_classes':self.number_of_classes}

